# Instagram  후원 브랜드 크롤링

### 1. instagram Login

In [2]:
#초기 세팅 필요

user_id = "a"
session_id = "b"

In [3]:
## 인스타그램 로그인 후 session_id, user_id 를 얻는 과정.

# Chrome driver manager사용

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
import ssl
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import pickle
# from fake_useragent import UserAgent

# 아마 https 관련 기능, requests 요청할 때 https 사이트의 경우 필요하다고 함.
ssl._create_default_https_context = ssl._create_unverified_context


# user_id = "a"
# session_id = "b"


## 이후에 def, if __name__ == '__main__' 방식으로 업데이트 예정
def executor():
    chrome_options = webdriver.ChromeOptions()
    # UserAgent 부분
#     userAgent = UserAgent().random
    userAgent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
    chrome_options.add_argument(f'user-agent={userAgent}')
    chrome_options.add_argument('--incognito')
    return webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

# def exec_chrom():
#     options = Options()
#     userAgent = UserAgent().random
#     options.add_argument(f'user-agent={userAgent}')
#     options.add_argument('--incognito')
#     driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
#     driver.maximize_window()
#     driver.get(url)
#     return driver

def user_data(driver):
    global user_id, session_id
    
    # 02. 로그인
    #경로 지정 #동작 제어
    
    login_button = driver.find_element(by=By.XPATH,value = '//*[@class="_ab21"]/div[2]/div[3]/button[1]')
    login_button.click()
    print("click1")
    
    id_inp = driver.find_element(by=By.XPATH,value = '//*[@id="loginForm"]/div[1]/div[3]/div/label')
    id_inp.click()
    id_inp.send_keys()
    print("click2")
    

    pw_inp = driver.find_element(by=By.XPATH,value = '//*[@id="loginForm"]/div[1]/div[4]/div/label')
    pw_inp.click()
    pw_inp.send_keys("asdf7530**")
    print("click3")
    

    log_btn = driver.find_element(by=By.XPATH,value = '//*[@id="loginForm"]/div[1]/div[6]/button')
    log_btn.click()
    
    time.sleep(4)
    pickle.dump(driver.get_cookies(), open("instagram_cookies.pkl", "wb")) #최초 로그인 후 쿠키 저장시에만
    cookies = pickle.load(open("instagram_cookies.pkl", "rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)
        print(cookie)
        if(cookie['name'] == 'ds_user_id'):
            user_id = cookie['value']
        if(cookie['name'] == 'sessionid'):
            session_id = cookie['value']
            
    return user_id,session_id

    
def login():
    driver = executor()
    driver.set_window_size(414, 800) 	#브라우저 크기 414*800으로 고정
    driver.get('https://www.instagram.com') #인스타그램 웹 켜기
    driver.implicitly_wait(8) 	# 2초 대기
    
    user_id, session_id = user_data(driver)
    print(user_id)
    print(session_id)

# 
# act = ActionChains(dr)      #동작 명령어 지정
# act.send_keys_to_element(id_box, 'rooneei').send_keys_to_element(password_box, '').click(login_button).perform()     #아이디 입력, 비밀 번호 입력, 로그인 버튼 클릭 수행
# time.sleep(2)



---

## 2. 태그 검색을 통한 게시물 확인

In [5]:
#초기 세팅

url_list=[]

### 2-1. <http://smartstore.naver.com/{판매페이지 링크}> 파싱 def

In [4]:
import re

def extract_url_part(url):
    pattern = r'smartstore\.naver\.com/([^/?]+)'
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    return None

def extract_base_url(url):
    # Remove the protocol (http:// or https://) from the URL
    url_protocol = url.split('//')[0]
    url_without_protocol = url.split('//')[1]
    
    # Remove the path and query string by finding the first '/'
    base_url1 = url_without_protocol.split('/')[0]
    base_url2 = extract_url_part(url)
    
    return (url_protocol +"//" + base_url1 + "/" + base_url2)

# urls = [
#     "https://smartstore.naver.com/wishpedia/products/8575922466",
#     "https://smartstore.naver.com/wishpedia?nt_source=littly.lin&nt_medium=social",
#     "https://m.smartstore.naver.com/deasdfd/products/857dfss"
# ]

# for i in urls:
#     print(extract_base_url(i))

### 2-3 다중 링크를 위한 페이지 (리틀리(littly), 링크트리(linktree)) 에서의 smartstore 링크 파싱 def

In [6]:
import json

def check_littly(other_url):
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
    }

    r1 = requests.get(other_url, headers=headers)
    # if r1.status_code == 200:
    soup = BeautifulSoup(r1.text, 'html.parser')
    scripts = soup.find_all('script')
    
    for script_tag in scripts:
        script_content = script_tag.string

        # 리틀리 링크
        if script_content and script_content.strip().startswith('window.data'):
            script_json = json.loads(script_content.strip()[12:])
            for block in script_json['blocks']:
                if 'url' in block:
                    check_targeturl_smartstore(block['url'],"smartstore.naver.com")
        
        #인스타비오 링크 - 귀찮아서 패스
#         if script_content and script_content.strip().startswith('window.__data'):
#             script_json = script_content.strip()[14:]
#             pprint(script_json)
#             if script_json['content']['cmpts']['links']['link']:
#                 check_targeturl_smartstore(script_json['content']['cmpts']['links']['link'],"smartstore.naver.com")
                
# https://instabio.cc/3113014svmsmC
# https://litt.ly/betterpresent
                
# check_otherlink("https://litt.ly/betterpresent")
    

In [7]:
def check_linktree(linktree_url):
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
    }

    r1 = requests.get(linktree_url, headers=headers)
    # if r1.status_code == 200:
    soup = BeautifulSoup(r1.text, 'html.parser')
    script = soup.find('body').find('script', id = '__NEXT_DATA__')
    
    r2 = json.loads(script.text)
    links = r2['props']['pageProps']['account']['links']
    
#     print(links[0]['url'])
    
    for link in links:
        check_targeturl_smartstore(link['url'],"smartstore.naver.com")
        

# check_linktree("https://linktr.ee/marcoroho")

### 2-2 해당 url이 smartstore url 인지 판별 (redirect 포함) + url_list 에 추가 / def

In [5]:
def check_targeturl_smartstore(external_url, target_string):
    if external_url and ( ( target_string in external_url ) or ( check_redirect_contains(external_url,target_string) ) ):
        if check_redirect_contains(external_url,target_string):
            external_url = check_redirect_contains(external_url,target_string)

        external_url = extract_base_url(external_url)
        
        if not (external_url in url_list):
            print(f"Smart Store url : {external_url}")
            url_list.append(external_url)
    else:
        print("XXXXXXXXXXXXXXXXXXXX", end = "\n\n\n")

### 2-4 태그 검색을 통한 게시물 확인

In [8]:
# 태그를 입력했을 때, 나오는 게시물들에 관한 json
# header 에서 2개 : Cookie(user_id, Session_id) 와 X-Ig-App-Id' 는 필수로 들어가야 함.)


import requests
from pprint import pprint

# url_list =[]

headers = {
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cookie' : f'ds_user_id={user_id}; sessionid={session_id};"',
    'X-Ig-App-Id': '936619743392459',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

headers2 = {
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'Cookie' : f'ds_user_id={user_id}; sessionid={session_id};"',
        'X-Ig-App-Id': '936619743392459',
        'User-Agent': "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
    }

def ready(tag_name,user_id,session_id):
    
#     tag_name = "후원"

    base_url = f"https://www.instagram.com/api/v1/tags/web_info/?tag_name={tag_name}"
    headers = {
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'Cookie' : f'ds_user_id={user_id}; sessionid={session_id};"',
        'X-Ig-App-Id': '936619743392459',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    }
    headers2 = {
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'Cookie' : f'ds_user_id={user_id}; sessionid={session_id};"',
        'X-Ig-App-Id': '936619743392459',
        'User-Agent': "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
    }
    
    response= requests.get(base_url,headers=headers)

    res = response.json()

    # 브라우저에 보이는 9개는 따로 json 파일의 최상단 중, data의 ['top']부분에 저장되어 있음.
    # ['sections'] 부분에 각각 3개씩 한묶음으로 해서 3개의 열이 json 파일로 보관되어 있음.

    all_count = res['count']
    root = res['data']['top']['sections']#[0],[1],[2] # section 다음에 리스트 0~2(브라우저에 보이는 3열)
    # tree1 = root['layout_content']['medias'] # medias 다음에 리스트 0~2(1열, 2열, 3열)
    
    # 브라우저에 보이지 않는 부분들
    invisible_root=res['data']['recent']['sections']# [0] # 여기까지 리스트 ({전체-9개}를 3으로 나눈 값만큼만 유효한 값)
    print(f"invisible_root 개수 : {len(invisible_root)}")
    # other_tree1=other_root['layout_content']['medias'] # 여기까지 리스트 (3개)
    
    return res,root,invisible_root

def check_redirect_contains(url, target_string):
    response = requests.get(url, allow_redirects=False)
    redirected_url = response.headers.get('Location')
    if redirected_url and target_string in redirected_url:
        return redirected_url
    return False

def list_execute(root,all_count,custom_limit):
    global i
    limit = all_count-9
    if all_count <= 9:
        limit = all_count
        
#     custom_limit = 20 # -> 매개변수로 바꿈
    break_check = False
    count = 0
    for post_list in root:
        count+=1
        print(f"count : {count}")
        if(break_check):
            break
        for tree in post_list['layout_content']['medias']: # 여기까지 리스트 (3개)
            if ((i > limit) or (i> custom_limit)):
                print("리미트로 인한 break")
                break_check = True
                break
            else:
                print (f"------------{i}번째 게시물--------------")
                i+=1
                tag_code = tree['media']['code']
                tag_user = tree['media']['caption']['user']['username']

                print(f"크롤링 게시물 url : https://www.instgram.com/p/{tag_code}")

                # 게시물 올린 사람의 인스타 계정으로 이동
                if tag_user not in user_list:
                    user_list.append(tag_user)
                    user_url = f'https://www.instagram.com/api/v1/users/web_profile_info/?username={tag_user}'
                    print(f"게시자 url : {user_url}")
                    try:
                        user_response= requests.get(user_url,headers=headers) # 에러 발생시 html 코드 나옴
                        external_url = user_response.json()['data']['user']['external_url'] # 여기서 jsonDecodeError 발생
                    except:
                        print("에러한번 발생")
    #                     login()
                        user_response= requests.get(user_url,headers=headers2)
                        external_url = user_response.json()['data']['user']['external_url']

                    # 그 인스타 계정의 다른 외부 주소가 Naver Smart Store 인 경우에만 접속.
                    target_string = "smartstore.naver.com"

                    if external_url and external_url.startswith("https://litt.ly/"):
                        check_littly(external_url)
                    elif external_url and external_url.startswith("https://linktr.ee/"):
                        check_linktree(external_url)
                    else:
                        check_targeturl_smartstore(external_url, target_string)    

def list(res,root,invisible_root, custom_limit):
    global i
    all_count = res['count']
    print(f"전체 게시물 : {all_count}")
#     i=1
    list_execute(root,all_count,custom_limit)
    if custom_limit > 9:
        list_execute(invisible_root,all_count,custom_limit)
        
    
#     for view_list in root:
#         for tree in view_list['layout_content']['medias']:
#             print (f"------------{i}번째 게시물--------------")
#             i+=1
#             tag_code = tree['media']['code']
#             tag_user = tree['media']['caption']['user']['username']
#             print(f"크롤링 게시물 url : https://www.instgram.com/p/{tag_code}")
            
#             # 게시물 올린 사람의 인스타 계정으로 이동
#             user_url = f'https://www.instagram.com/api/v1/users/web_profile_info/?username={tag_user}'
#             print(f"게시자 url : {user_url}")
#             try:
#                 user_response= requests.get(user_url,headers=headers)
#             except:
#                 main()
#                 user_response= requests.get(user_url,headers=headers)
            
#             # 그 인스타 계정의 다른 외부 주소가 Naver Smart Store 인 경우에만 접속.
#             external_url = user_response.json()['data']['user']['external_url']
#             target_string = "smartstore.naver.com"
            
#             check_targeturl_smartstore(external_url, target_string)
            
# #             if external_url and ( ( target_string in external_url ) or ( check_redirect_contains(external_url,target_string) ) ):
# #                 if check_redirect_contains(external_url,target_string):
# #                     external_url = check_redirect_contains(external_url,target_string)
# #                 if not (external_url in url_list):
# #                     print(f"Smart Store url : {external_url}")
# #                     url_list.append(external_url)
# #             else:
# #                 print("XXXXXXXXXXXXXXXXXXXX", end = "\n\n\n")
                
#     print("-----------=====-보이지 않는 게시물------=====------------")
    
#     limit = all_count-9
# #     custom_limit = 20 # -> 매개변수로 바꿈
#     break_check = False
#     count = 0
#     for invisible_list in invisible_root:
#         if(break_check):
#             break
#         for tree in invisible_list['layout_content']['medias']: # 여기까지 리스트 (3개)
#             if ((i > limit) or (i> custom_limit)):
#                 break_check = True
#                 break
#             else:
#                 print (f"------------{i}번째 게시물--------------")
#                 i+=1
#                 tag_code = tree['media']['code']
#                 tag_user = tree['media']['caption']['user']['username']
#                 print(f"크롤링 게시물 url : https://www.instgram.com/p/{tag_code}")

#                 # 게시물 올린 사람의 인스타 계정으로 이동
#                 user_url = f'https://www.instagram.com/api/v1/users/web_profile_info/?username={tag_user}'
#                 print(f"게시자 url : {user_url}")
#                 try:
#                     user_response= requests.get(user_url,headers=headers)
#                 except:
#                     main()
#                     user_response= requests.get(user_url,headers=headers)

#                 # 그 인스타 계정의 다른 외부 주소가 Naver Smart Store 인 경우에만 접속.
#                 external_url = user_response.json()['data']['user']['external_url']
#                 target_string = "smartstore.naver.com"
                
#                 check_targeturl_smartstore(external_url, target_string)

# #                 if external_url and ( ( target_string in external_url ) or ( check_redirect_contains(external_url,target_string) ) ):
# #                     if check_redirect_contains(external_url,target_string):
# #                         external_url = check_redirect_contains(external_url,target_string)
# #                     if not (external_url in url_list):
# #                         print(f"Smart Store url : {external_url}")
# #                         url_list.append(external_url)
# #                 else:
# #                     print("XXXXXXXXXXXXXXXXXXXX", end = "\n\n\n")
            


---

## 3. 스마트스토어 json file  

In [9]:
import requests
from bs4 import BeautifulSoup
import re
import json
from pprint import *


# def parse_url(url):
#     parts = url.split("/")
    
#     if len(parts) >= 4:
#         new_url = "/".join(parts[:4]) + "/"
#         return new_url
#     else:
#         return url
#################################### -> extract_base_url 로 대체

def remove_emoji(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    
    return re.sub(emoj, '', data)


def smartstore_json(url):
    
    smartstore_url = extract_base_url(url)

    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
    }

    r1 = requests.get(smartstore_url, headers=headers)
    # if r1.status_code == 200:
    soup = BeautifulSoup(r1.text, 'html.parser')
    scripts = soup.find('body').find_all('script')

#     cnt = 0
#     for script in scripts:
#         cnt += 1

    data = scripts[0].text.strip()[27:]


    remove_emoji(data)
    r2 = json.loads(data)
#     pprint(r2)  # <class 'dict'>
    return r2

In [10]:
from pprint import *

def item(res2,url):
    item_cnt = 0
    for dict_elem in res2['widgetContents']['wholeProductWidget']['A']['data']['simpleProducts']:  # 상품전체 부분을 대상으로 함
        name = dict_elem['name']
        items ={
            "pname": dict_elem["name"],
            "purl" : f'{url}/products/{dict_elem["id"]}',
            "pimg": dict_elem["representativeImageUrl"],
            "pcategory1" : dict_elem["category"]["wholeCategoryName"],
            "pcategory2" : dict_elem["category"]["categoryName"],
            "psalePrice": dict_elem["salePrice"],
            "pdiscountedRatio": dict_elem["benefitsView"]["discountedRatio"],
            "pdiscountedSalePrice": dict_elem["benefitsView"]["discountedSalePrice"],
            "pbaseFee" : dict_elem['productDeliveryInfo']['baseFee'],
            "bname": dict_elem["channel"]["channelName"]
            
#             "bintro": dict_elem["channel"]['description'],
#             "bimg" : dict_elem['representImageInfoList']['imageUrl'],
            
#             "productStatusType": dict_elem["productStatusType"],
            
            
#             "textReviewPoint": dict_elem["benefitsView"]["textReviewPoint"],
#             "photoVideoReviewPoint": dict_elem["benefitsView"]["photoVideoReviewPoint"],
#             "reviewCount": dict_elem["reviewAmount"]["totalReviewCount"],
#             "averageReviewScore": dict_elem["reviewAmount"]["averageReviewScore"],
            
        }
        item_list.append(items)
        pprint(item_list[item_cnt])
        item_cnt += 1
    
    return item_list

In [11]:
def brand(res2,url):
    dict_elem = res2['smartStoreV2']['channel']  # 상품전체 부분을 대상으로 함
    brands ={            
        "bname": dict_elem["channelName"],
        "bintro": dict_elem['description'],
        "bimg" : dict_elem['representativeImageUrl'],
        "burl" : url
        
#         "productStatusType": dict_elem["productStatusType"],


#         "textReviewPoint": dict_elem["benefitsView"]["textReviewPoint"],
#         "photoVideoReviewPoint": dict_elem["benefitsView"]["photoVideoReviewPoint"],
#         "reviewCount": dict_elem["reviewAmount"]["totalReviewCount"],
#         "averageReviewScore": dict_elem["reviewAmount"]["averageReviewScore"],
            
    }
    brand_list.append(brands)
    
    return brand_list

# Main 코드

In [12]:
def get_post(query,custom_limit):
    response,root,invisible_root = ready(query,user_id,session_id) # 각 게시물 별 root 추가
    list(response,root,invisible_root,custom_limit) # 게시물 순 리스트 출력, url_list 에 각 스마트스토어 리스트 추가

In [169]:
from pprint import *

if __name__ == '__main__':
    global user_id
    global session_id    
    global url_list
#     global url_list = ['https://m.smartstore.naver.com/de_lemonmarket',
#  'https://smartstore.naver.com/puffy_thing/',
#  'http://branddaon.com/']
    global item_list
    global brand_list
    global i
    global user_list
    
    user_id = "a"
    session_id = "b"
    url_list = []
    item_list = []
    brand_list = []
    user_list=[]
    i=1

    headers = {
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'Cookie' : f'ds_user_id={user_id}; sessionid={session_id};"',
        'X-Ig-App-Id': '936619743392459',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    }

    login() # user_id, session_id 생성
    query_list = ["후원브랜드","후원팔찌","유기견후원"]
    
    for query in query_list:
        i=1
        get_post(query,40) #url_list 추가
    
    # 각 smartstore url 의 json에서 모든 상품 조회 - item()
    # 같은 json 에서 브랜드까지 추출 - brand()
    for url in url_list:
        res2 = smartstore_json(url) # smartstore url 로부터 json 추출
        item(res2,url) # item_list 에 각 item 별 json 데이터 dicctionary 에 추가
        brand(res2,url) # brand_list

click1
click2
click3
{'domain': '.instagram.com', 'httpOnly': True, 'name': 'rur', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': '"EAG\\0549678042146\\0541721454328:01f7a475f21c403baa5f62372087b22398641803f6654b1c74ae30392c8226827ef5f000"'}
{'domain': '.instagram.com', 'expiry': 1690523126, 'httpOnly': True, 'name': 'shbid', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': '"2588\\0549678042146\\0541721454327:01f78befc8e3297cb135496117ff68c945e793ee609731bc7d6db34a7561966f3d4f7475"'}
{'domain': '.instagram.com', 'expiry': 1690523126, 'httpOnly': True, 'name': 'shbts', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': '"1689918327\\0549678042146\\0541721454327:01f7ee77027fe27202092daddd58573c29a0872684de5c692c6cabba66a606d84162326a"'}
{'domain': '.instagram.com', 'expiry': 1721454326, 'httpOnly': True, 'name': 'sessionid', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': '9678042146%3AobeTSan77EkEIx%3A24%3AAYfYlVSpZY-ufKX53eu2fEUAsgayYUUAlyiZvj2JDA'}
{'

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [43]:
brand_list

[{'bname': '드 르몽마켓',
  'bintro': '세상을 따사롭게 비추는 햇살 한 줄기가 되겠습니다.',
  'bimg': 'https://shop-phinf.pstatic.net/20220107_182/1641534641455CbsDG_JPEG/42670530164445771_1378069890.jpg',
  'burl': 'https://m.smartstore.naver.com/de_lemonmarket'},
 {'bname': '퍼피띵',
  'bintro': '당신의 손길로 유기견의 세상을 바꿀 수 있습니다',
  'bimg': 'https://shop-phinf.pstatic.net/20220615_286/1655268390383uadOH_JPEG/56404223964483283_487881194.jpg',
  'burl': 'https://smartstore.naver.com/puffy_thing'},
 {'bname': '다온 :DAON',
  'bintro': '다온 :DAON',
  'bimg': 'https://shop-phinf.pstatic.net/20230513_73/1683984489849zmMOB_PNG/85120385579642389_1495267821.png',
  'burl': 'https://smartstore.naver.com/trendyttt'},
 {'bname': '다다멍냥',
  'bintro': '다다멍냥 유기동물과 후원 주얼리 브랜드',
  'bimg': 'https://shop-phinf.pstatic.net/20211014_264/1634199167983ROs9M_JPEG/35335002815500815_172696851.jpg',
  'burl': 'https://smartstore.naver.com/heelaser'},
 {'bname': '묘댕이네-bona',
  'bintro': '모든 수익금은 유기동물을 위해 사용합니다. 패션잡화점 비즈매듭공예',
  'bimg': 'https://shop-

In [44]:
url_list

['https://m.smartstore.naver.com/de_lemonmarket',
 'https://smartstore.naver.com/puffy_thing',
 'https://smartstore.naver.com/trendyttt',
 'https://smartstore.naver.com/heelaser',
 'https://smartstore.naver.com/myodeng',
 'https://smartstore.naver.com/ppottuppttu',
 'https://smartstore.naver.com/gfound',
 'https://m.smartstore.naver.com/petsdaily',
 'https://smartstore.naver.com/waterstar',
 'https://smartstore.naver.com/kongdahouse',
 'https://smartstore.naver.com/pawinhand',
 'https://smartstore.naver.com/peticion_1214',
 'https://m.smartstore.naver.com/unimpressedhusky',
 'https://smartstore.naver.com/moii',
 'https://smartstore.naver.com/hisdog']

# Main2 - 크롤링 요청

In [ ]:

headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
#     'Content-Length': '500',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cookie' : f'ds_user_id={user_id}; sessionid={session_id};"',
    'X-Ig-App-Id': '936619743392459',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36.36',
#     'Referer': 'https://www.instagram.com/p/CuyTmHAve9O/?img_index=1',
#     https://www.instagram.com/p/Cui-tg-yD2o/?img_index=1
#     https://www.instagram.com/p/CuyTmHAve9O/?img_index=1
# https://www.instagram.com/p/CrxJRgipifB/
    'X-Csrftoken':'kFIGJsZ1zXpm1i0lMUOYeCC1lKadWVwX',
    'Content-Type':'application/x-www-form-urlencoded'
}


user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:94.0) Gecko/20100101 Firefox/94.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_5_2) AppleWebKit/537.36 (KHTML, like Gecko) Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36 Edg/96.0.1054.34",
    "Opera/9.80 (Windows NT 6.1; WOW64) Presto/2.12.388 Version/12.18",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36"
]

# headers = {
#     'Accept': '*/*',
#     'Accept-Encoding': 'gzip, deflate, br',
#     'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
#     'Content-Length': '265',
#     'Content-Type': 'application/x-www-form-urlencoded',
#     'Cookie': 'ds_user_id=9678042146; csrftoken=kFIGJsZ1zXpm1i0lMUOYeCC1lKadWVwX; dpr=1.25; mid=ZLODdQALAAFRfdCrwsL66F9-ZK7l; ig_did=5E2AEEF7-8249-4439-B4CB-1E311489B6D7; datr=dYOzZDXmg4nwuqqEAdaNac0x; fbm_124024574287414=base_domain=.instagram.com; shbid="2588\0549678042146\0541721288269:01f7fbc5faad6f2b1bea337f9d47c2e5bbba02818caad89b347ca3056c782beb9f7a929c"; shbts="1689752269\0549678042146\0541721288269:01f7dde54b9390092d279a4d6b404e69f0e1f65843705844ecfe586b3f9dcd92c1ec8ef4"; sessionid=9678042146%3AqHmZC2H5Bjdnlb%3A10%3AAYeJ3haAYwwXB1kfzRf76Inm1r-pCTPWJw1Lykbw0YE; rur="EAG\0549678042146\0541721408754:01f7e8ce0c4faa6d630a069ebb03969bcd417b92ead7be3811dbcef4e62671759669623c"',
#     'Origin': 'https://www.instagram.com',
#     'Referer': 'https://www.instagram.com/p/CuNeZckyjqM/?img_index=1',
#     'Sec-Ch-Prefers-Color-Scheme': 'light',
#     'Sec-Ch-Ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
#     'Sec-Ch-Ua-Full-Version-List': '"Not.A/Brand";v="8.0.0.0", "Chromium";v="114.0.5735.199", "Google Chrome";v="114.0.5735.199"',
#     'Sec-Ch-Ua-Mobile': '?0',
#     'Sec-Ch-Ua-Platform': '"Windows"',
#     'Sec-Ch-Ua-Platform-Version': '"10.0.0"',
#     'Sec-Fetch-Dest': 'empty',
#     'Sec-Fetch-Mode': 'cors',
#     'Sec-Fetch-Site': 'same-origin',
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
#     'Viewport-Width': '818',
#     'X-Asbd-Id': '129477',
#     'X-Csrftoken': 'kFIGJsZ1zXpm1i0lMUOYeCC1lKadWVwX',
#     'X-Ig-App-Id': '936619743392459',
#     'X-Ig-Www-Claim': 'hmac.AR220AKok4v1vxARXqZ6BbKkYf9kgjej9zQCryIlTk8LXkAn',
#     'X-Instagram-Ajax': '1007873689',
#     'X-Requested-With': 'XMLHttpRequest'
# }

# params = {
# #     'include_persistent': '0',
#     'max_id': 'QVFCTW95M1JHZlphS3J1d0pxakRlMVlsOWdwTnM2Z3l1dlpiR09GalZEb0dJRFlkUkhJQ0RUSm1maVRPZVR1bUlLMFJ5Sld2NEZoYkVIc0ZOYUxvVk9HMA==',
#     'next_media_ids[]': ['asdfadf', 'asdfasdf'],
#     'page': '5',
# #     'surface': 'grid',
# #     'tab': 'recent'
# }

user_list=[]


query_list = ["후원브랜드","후원팔찌","유기견후원"]
for query in query_list:
    url_test = f"https://www.instagram.com/api/v1/tags/{query}/sections/"
    print(query)
    for kk in range(2):
        print(kk)
        res11 = requests.post(url_test, headers=headers,data=params)

        pprint(len(json.loads(res11.text)['sections']))
        sse = json.loads(res11.text)['sections']

        i=1
        for j in sse:
            ssk = j['layout_content']['medias']
            for k in ssk:
                post_code = k['media']['code']
                try:
                    post_user = k['media']['caption']['user']['username']
                except:
                    break
                print(post_code)
                if post_code in code_list:
                    print("중복발생")
                else:
                    code_list.append(post_code)
                    
                if post_user not in user_list:
                    user_list.append(post_user)
                    user_url = f'https://www.instagram.com/api/v1/users/web_profile_info/?username={post_user}'
                    print(f"게시자 url : {user_url}")
                    try:
                        user_response= requests.get(user_url,headers=headers) # 에러 발생시 html 코드 나옴
                        external_url = user_response.json()['data']['user']['external_url'] # 여기서 jsonDecodeError 발생
                    except:
                        for user_agent in user_agents:
                            headers['User-Agent'] = user_agent
                            try:
                                user_response= requests.get(user_url,headers=headers)
                                break
                            except json.JSONDecodeError:
                                # 에러가 발생하면 다음 User-Agent로 시도
                                print("에러한번 발생")
                                continue

                    # 그 인스타 계정의 다른 외부 주소가 Naver Smart Store 인 경우에만 접속.
                    target_string = "smartstore.naver.com"

                    if external_url and external_url.startswith("https://litt.ly/"):
                        check_littly(external_url)
                    elif external_url and external_url.startswith("https://linktr.ee/"):
                        check_linktree(external_url)
                    else:
                        check_targeturl_smartstore(external_url, target_string)    


In [2]:
url_list

NameError: name 'url_list' is not defined

In [200]:
item_list=[]
brand_list=[]

for url in url_list:
    res2 = smartstore_json(url) # smartstore url 로부터 json 추출
    item(res2,url) # item_list 에 각 item 별 json 데이터 dicctionary 에 추가
    brand(res2,url) # brand_list

{'bname': '히즈독',
 'pbaseFee': 3500,
 'pcategory1': '생활/건강>반려동물>강아지 간식>수제간식',
 'pcategory2': '수제간식',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 19800,
 'pimg': 'https://shop-phinf.pstatic.net/20230521_249/1684625490682NOWXl_JPEG/37993035503892655_1853968805.jpg',
 'pname': '히즈독 강아지 스테이크 영양간식 4종',
 'psalePrice': 19800,
 'purl': 'https://smartstore.naver.com/hisdog/products/8584151309'}
{'bname': '히즈독',
 'pbaseFee': 3500,
 'pcategory1': '생활/건강>반려동물>강아지 간식>수제간식',
 'pcategory2': '수제간식',
 'pdiscountedRatio': 65,
 'pdiscountedSalePrice': 6900,
 'pimg': 'https://shop-phinf.pstatic.net/20230130_12/1675007777552ixbog_JPEG/76143611370274110_1034006743.jpg',
 'pname': '강아지 수제간식 히즈독 한우등뼈, 닭가슴살, 돼지육포 모음전',
 'psalePrice': 20000,
 'purl': 'https://smartstore.naver.com/hisdog/products/7995297503'}
{'bname': '히즈독',
 'pbaseFee': 0,
 'pcategory1': '생활/건강>반려동물>리빙용품>계단/스텝',
 'pcategory2': '계단/스텝',
 'pdiscountedRatio': 63,
 'pdiscountedSalePrice': 65000,
 'pimg': 'https://shop-phinf.pstatic.net/2022090

{'bname': '히즈독',
 'pbaseFee': 3500,
 'pcategory1': '생활/건강>반려동물>강아지 간식>수제간식',
 'pcategory2': '수제간식',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 19800,
 'pimg': 'https://shop-phinf.pstatic.net/20230521_249/1684625490682NOWXl_JPEG/37993035503892655_1853968805.jpg',
 'pname': '히즈독 강아지 스테이크 영양간식 4종',
 'psalePrice': 19800,
 'purl': 'https://smartstore.naver.com/hisdog/products/8584151309'}
{'bname': '히즈독',
 'pbaseFee': 3500,
 'pcategory1': '생활/건강>반려동물>강아지 간식>수제간식',
 'pcategory2': '수제간식',
 'pdiscountedRatio': 65,
 'pdiscountedSalePrice': 6900,
 'pimg': 'https://shop-phinf.pstatic.net/20230130_12/1675007777552ixbog_JPEG/76143611370274110_1034006743.jpg',
 'pname': '강아지 수제간식 히즈독 한우등뼈, 닭가슴살, 돼지육포 모음전',
 'psalePrice': 20000,
 'purl': 'https://smartstore.naver.com/hisdog/products/7995297503'}
{'bname': '히즈독',
 'pbaseFee': 0,
 'pcategory1': '생활/건강>반려동물>리빙용품>계단/스텝',
 'pcategory2': '계단/스텝',
 'pdiscountedRatio': 63,
 'pdiscountedSalePrice': 65000,
 'pimg': 'https://shop-phinf.pstatic.net/2022090

{'bname': '히즈독',
 'pbaseFee': 3500,
 'pcategory1': '생활/건강>반려동물>강아지 간식>수제간식',
 'pcategory2': '수제간식',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 19800,
 'pimg': 'https://shop-phinf.pstatic.net/20230521_249/1684625490682NOWXl_JPEG/37993035503892655_1853968805.jpg',
 'pname': '히즈독 강아지 스테이크 영양간식 4종',
 'psalePrice': 19800,
 'purl': 'https://smartstore.naver.com/hisdog/products/8584151309'}
{'bname': '히즈독',
 'pbaseFee': 3500,
 'pcategory1': '생활/건강>반려동물>강아지 간식>수제간식',
 'pcategory2': '수제간식',
 'pdiscountedRatio': 65,
 'pdiscountedSalePrice': 6900,
 'pimg': 'https://shop-phinf.pstatic.net/20230130_12/1675007777552ixbog_JPEG/76143611370274110_1034006743.jpg',
 'pname': '강아지 수제간식 히즈독 한우등뼈, 닭가슴살, 돼지육포 모음전',
 'psalePrice': 20000,
 'purl': 'https://smartstore.naver.com/hisdog/products/7995297503'}
{'bname': '히즈독',
 'pbaseFee': 0,
 'pcategory1': '생활/건강>반려동물>리빙용품>계단/스텝',
 'pcategory2': '계단/스텝',
 'pdiscountedRatio': 63,
 'pdiscountedSalePrice': 65000,
 'pimg': 'https://shop-phinf.pstatic.net/2022090

{'bname': '히즈독',
 'pbaseFee': 3500,
 'pcategory1': '생활/건강>반려동물>강아지 간식>수제간식',
 'pcategory2': '수제간식',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 19800,
 'pimg': 'https://shop-phinf.pstatic.net/20230521_249/1684625490682NOWXl_JPEG/37993035503892655_1853968805.jpg',
 'pname': '히즈독 강아지 스테이크 영양간식 4종',
 'psalePrice': 19800,
 'purl': 'https://smartstore.naver.com/hisdog/products/8584151309'}
{'bname': '히즈독',
 'pbaseFee': 3500,
 'pcategory1': '생활/건강>반려동물>강아지 간식>수제간식',
 'pcategory2': '수제간식',
 'pdiscountedRatio': 65,
 'pdiscountedSalePrice': 6900,
 'pimg': 'https://shop-phinf.pstatic.net/20230130_12/1675007777552ixbog_JPEG/76143611370274110_1034006743.jpg',
 'pname': '강아지 수제간식 히즈독 한우등뼈, 닭가슴살, 돼지육포 모음전',
 'psalePrice': 20000,
 'purl': 'https://smartstore.naver.com/hisdog/products/7995297503'}
{'bname': '히즈독',
 'pbaseFee': 0,
 'pcategory1': '생활/건강>반려동물>리빙용품>계단/스텝',
 'pcategory2': '계단/스텝',
 'pdiscountedRatio': 63,
 'pdiscountedSalePrice': 65000,
 'pimg': 'https://shop-phinf.pstatic.net/2022090

{'bname': '히즈독',
 'pbaseFee': 3500,
 'pcategory1': '생활/건강>반려동물>강아지 간식>수제간식',
 'pcategory2': '수제간식',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 19800,
 'pimg': 'https://shop-phinf.pstatic.net/20230521_249/1684625490682NOWXl_JPEG/37993035503892655_1853968805.jpg',
 'pname': '히즈독 강아지 스테이크 영양간식 4종',
 'psalePrice': 19800,
 'purl': 'https://smartstore.naver.com/hisdog/products/8584151309'}
{'bname': '히즈독',
 'pbaseFee': 3500,
 'pcategory1': '생활/건강>반려동물>강아지 간식>수제간식',
 'pcategory2': '수제간식',
 'pdiscountedRatio': 65,
 'pdiscountedSalePrice': 6900,
 'pimg': 'https://shop-phinf.pstatic.net/20230130_12/1675007777552ixbog_JPEG/76143611370274110_1034006743.jpg',
 'pname': '강아지 수제간식 히즈독 한우등뼈, 닭가슴살, 돼지육포 모음전',
 'psalePrice': 20000,
 'purl': 'https://smartstore.naver.com/hisdog/products/7995297503'}
{'bname': '히즈독',
 'pbaseFee': 0,
 'pcategory1': '생활/건강>반려동물>리빙용품>계단/스텝',
 'pcategory2': '계단/스텝',
 'pdiscountedRatio': 63,
 'pdiscountedSalePrice': 65000,
 'pimg': 'https://shop-phinf.pstatic.net/2022090

{'bname': '히즈독',
 'pbaseFee': 3500,
 'pcategory1': '생활/건강>반려동물>강아지 간식>수제간식',
 'pcategory2': '수제간식',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 19800,
 'pimg': 'https://shop-phinf.pstatic.net/20230521_249/1684625490682NOWXl_JPEG/37993035503892655_1853968805.jpg',
 'pname': '히즈독 강아지 스테이크 영양간식 4종',
 'psalePrice': 19800,
 'purl': 'https://smartstore.naver.com/hisdog/products/8584151309'}
{'bname': '히즈독',
 'pbaseFee': 3500,
 'pcategory1': '생활/건강>반려동물>강아지 간식>수제간식',
 'pcategory2': '수제간식',
 'pdiscountedRatio': 65,
 'pdiscountedSalePrice': 6900,
 'pimg': 'https://shop-phinf.pstatic.net/20230130_12/1675007777552ixbog_JPEG/76143611370274110_1034006743.jpg',
 'pname': '강아지 수제간식 히즈독 한우등뼈, 닭가슴살, 돼지육포 모음전',
 'psalePrice': 20000,
 'purl': 'https://smartstore.naver.com/hisdog/products/7995297503'}
{'bname': '히즈독',
 'pbaseFee': 0,
 'pcategory1': '생활/건강>반려동물>리빙용품>계단/스텝',
 'pcategory2': '계단/스텝',
 'pdiscountedRatio': 63,
 'pdiscountedSalePrice': 65000,
 'pimg': 'https://shop-phinf.pstatic.net/2022090

In [1]:
url_list

NameError: name 'url_list' is not defined

### json 형식으로 데이터 Spring boot 서버에 전송

In [65]:
import json
import requests

# # 딕셔너리 생성
# data = {
#     "name": "John",
#     "age": 30,
#     "email": "john@example.com"
# }

item = {
    "item":
}
for item_json in item_list:
    # 딕셔너리를 JSON 형식으로 변환
#     json_data = json.dumps(item_json)
    pprint(item_json)

    # Spring Boot 서버 URL
    url = "http://localhost:8080/test/item"

    # POST 요청 전송
    response = requests.post(url, json=item_json)

    if response.status_code == 200:
        result = response.json()
        print(f'User created with ID {result["id"]}')
    else:
        print(f'Request failed with status code {response.status_code}')

{'averageReviewScore': 4.96,
 'categoryId': '50004155',
 'categoryName': '패션잡화>주얼리>반지>패션반지',
 'channelName': '드 르몽마켓',
 'discountedRatio': 8,
 'discountedSalePrice': 15500,
 'id': 6720658283,
 'name': '하트 심장 박동 유기견 유기묘 동물 아동 후원 기부 커플 학생 우정 써지컬 링 반지',
 'photoVideoReviewPoint': 100,
 'productNo': 6688653575,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20230202_263/16753209836401AXa4_JPEG/76456811469995351_939037103.jpg',
 'reviewCount': 75,
 'salePrice': 17000,
 'textReviewPoint': 30}
Request failed with status code 200
{'averageReviewScore': 4.94,
 'categoryId': '50004155',
 'categoryName': '패션잡화>주얼리>반지>패션반지',
 'channelName': '드 르몽마켓',
 'discountedRatio': 8,
 'discountedSalePrice': 15500,
 'id': 6263624235,
 'name': '생명 레터링 유기견 유기묘 동물 아동 후원 기부 커플 학생 우정 각인 써지컬 링 반지',
 'photoVideoReviewPoint': 100,
 'productNo': 6233893568,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20220205_135/1644044536659Wgqpp_J

Request failed with status code 200
{'averageReviewScore': 0,
 'categoryId': '50000850',
 'categoryName': '가구/인테리어>수예>수예용품/부자재',
 'channelName': '퍼피띵',
 'discountedRatio': 25,
 'discountedSalePrice': 2900,
 'id': 8664498667,
 'name': '[5개세트] 유기견 후원 요술 단추 똑딱이 바지 허리 줄이기 늘리기 청바지 수선',
 'photoVideoReviewPoint': 250,
 'productNo': 8623609983,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20230606_187/1686025341075xcE8X_JPEG/2488228436228491_973696732.jpg',
 'reviewCount': 0,
 'salePrice': 3900,
 'textReviewPoint': 100}
Request failed with status code 200
{'averageReviewScore': 3,
 'categoryId': '50000826',
 'categoryName': '패션의류>여성언더웨어/잠옷>잠옷/홈웨어',
 'channelName': '퍼피띵',
 'discountedRatio': 27,
 'discountedSalePrice': 15900,
 'id': 8593059242,
 'name': '[무배/당일출고] 유기견 후원 심플 반팔 커플 우정 파자마 잠옷 세트',
 'photoVideoReviewPoint': 250,
 'productNo': 8552082216,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20230523_83/1

Request failed with status code 200
{'averageReviewScore': 0,
 'categoryId': '50000823',
 'categoryName': '패션의류>여성언더웨어/잠옷>브라',
 'channelName': '퍼피띵',
 'discountedRatio': 23,
 'discountedSalePrice': 9900,
 'id': 8207223680,
 'name': '[무배/당일출고] 심리스 노와이어 편한 뽕 보정 스포츠 면 브라 속옷',
 'photoVideoReviewPoint': 250,
 'productNo': 8168092313,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20230314_125/1678781467888J608B_JPEG/79917310596824246_806023912.jpg',
 'reviewCount': 0,
 'salePrice': 12900,
 'textReviewPoint': 100}
Request failed with status code 200
{'averageReviewScore': 2,
 'categoryId': '50000850',
 'categoryName': '가구/인테리어>수예>수예용품/부자재',
 'channelName': '퍼피띵',
 'discountedRatio': 23,
 'discountedSalePrice': 990,
 'id': 8136805806,
 'name': '[당일출고] 유기견 후원 요술 단추 똑딱이 바지 허리 줄이기 늘리기 청바지 수선',
 'photoVideoReviewPoint': 250,
 'productNo': 8097987537,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20230223_293/1677

In [64]:
import json

data = {
    "name": "하트 심장 박동 유기견 유기묘 동물 아동 후원 기부 커플 학생 우정 써지컬 링 반지",
    "id": 6720658283,
    "categoryId": "50004155",
    "categoryName": "패션잡화>주얼리>반지>패션반지",
    "channelName": "드 르몽마켓",
    "productNo": 6688653575,
    "salePrice": 17000,
    "productStatusType": "SALE",
    "discountedSalePrice": 15500,
    "discountedRatio": 8,
    "textReviewPoint": 30,
    "photoVideoReviewPoint": 100,
    "reviewCount": 75,
    "averageReviewScore": 4.96,
    "representativeImageUrl": "https://shop-phinf.pstatic.net/20230202_263/16753209836401AXa4_JPEG/76456811469995351_939037103.jpg"
}
# json_data = json.dumps(data, ensure_ascii=False)
pprint(data)

# Spring Boot 서버 URL
url = "http://localhost:8080/test/item"

# POST 요청 전송
response = requests.post(url, json=data)
if response.status_code == 200:
    result = response.json()
    print(result)

{'averageReviewScore': 4.96,
 'categoryId': '50004155',
 'categoryName': '패션잡화>주얼리>반지>패션반지',
 'channelName': '드 르몽마켓',
 'discountedRatio': 8,
 'discountedSalePrice': 15500,
 'id': 6720658283,
 'name': '하트 심장 박동 유기견 유기묘 동물 아동 후원 기부 커플 학생 우정 써지컬 링 반지',
 'photoVideoReviewPoint': 100,
 'productNo': 6688653575,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20230202_263/16753209836401AXa4_JPEG/76456811469995351_939037103.jpg',
 'reviewCount': 75,
 'salePrice': 17000,
 'textReviewPoint': 30}
{'isSuccess': True, 'code': 200, 'message': '요청에 성공하였습니다.', 'result': {'name': '하트 심장 박동 유기견 유기묘 동물 아동 후원 기부 커플 학생 우정 써지컬 링 반지', 'id': 6720658283, 'categoryId': '50004155', 'categoryName': '패션잡화>주얼리>반지>패션반지', 'channelName': '드 르몽마켓', 'productNo': 6688653575, 'salePrice': 17000, 'productStatusType': 'SALE', 'discountedSalePrice': 15500, 'discountedRatio': 8, 'textReviewPoint': 30, 'photoVideoReviewPoint': 100, 'reviewCount': 75, 'averageReviewScore': 4.96, 'repres

In [53]:
# 딕셔너리 생성
data = {
    "name": "John",
    "age": 30,
    "email": "john@example.com"
}
data

{'name': 'John', 'age': 30, 'email': 'john@example.com'}

In [216]:
user_response.json()

NameError: name 'user_response' is not defined

In [244]:
url_list


['https://smartstore.naver.com/puffy_thing',
 'https://m.smartstore.naver.com/de_lemonmarket',
 'https://smartstore.naver.com/trendyttt',
 'https://smartstore.naver.com/heelaser',
 'https://smartstore.naver.com/myodeng']

In [259]:
for url in url_list:
    res2 = smartstore_json(url)
    item(res2) # item_list 에 각 item 별 json 데이터 dicctionary 에 추가

{'averageReviewScore': 4.84,
 'categoryId': '50004174',
 'categoryName': '패션잡화>주얼리>목걸이>패션목걸이',
 'channelName': '퍼피띵',
 'discountedRatio': 40,
 'discountedSalePrice': 11900,
 'id': 7044991880,
 'name': '[무배/당일출고] 유기견후원 써지컬목걸이 기부 물방울 목걸이',
 'photoVideoReviewPoint': 250,
 'productNo': 7011523432,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20220826_61/1661519380627EvF1H_JPEG/62655215423944826_1290644050.jpg',
 'reviewCount': 209,
 'salePrice': 19900,
 'textReviewPoint': 100}
{'averageReviewScore': 4.91,
 'categoryId': '50004194',
 'categoryName': '패션잡화>주얼리>팔찌>패션팔찌',
 'channelName': '퍼피띵',
 'discountedRatio': 13,
 'discountedSalePrice': 12900,
 'id': 6803849415,
 'name': '유기견후원팔찌 기부팔찌 학생 커플 팔찌',
 'photoVideoReviewPoint': 250,
 'productNo': 6771467935,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20230205_171/1675599430670YVMOC_JPEG/76735326400102865_575503607.jpg',
 'reviewCount': 355,
 'salePrice': 14

{'averageReviewScore': 4.84,
 'categoryId': '50004174',
 'categoryName': '패션잡화>주얼리>목걸이>패션목걸이',
 'channelName': '퍼피띵',
 'discountedRatio': 40,
 'discountedSalePrice': 11900,
 'id': 7044991880,
 'name': '[무배/당일출고] 유기견후원 써지컬목걸이 기부 물방울 목걸이',
 'photoVideoReviewPoint': 250,
 'productNo': 7011523432,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20220826_61/1661519380627EvF1H_JPEG/62655215423944826_1290644050.jpg',
 'reviewCount': 209,
 'salePrice': 19900,
 'textReviewPoint': 100}
{'averageReviewScore': 4.91,
 'categoryId': '50004194',
 'categoryName': '패션잡화>주얼리>팔찌>패션팔찌',
 'channelName': '퍼피띵',
 'discountedRatio': 13,
 'discountedSalePrice': 12900,
 'id': 6803849415,
 'name': '유기견후원팔찌 기부팔찌 학생 커플 팔찌',
 'photoVideoReviewPoint': 250,
 'productNo': 6771467935,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20230205_171/1675599430670YVMOC_JPEG/76735326400102865_575503607.jpg',
 'reviewCount': 355,
 'salePrice': 14

{'averageReviewScore': 5,
 'categoryId': '50004025',
 'categoryName': '패션잡화>패션소품>스타킹>압박보정스타킹',
 'channelName': '퍼피띵',
 'discountedRatio': 53,
 'discountedSalePrice': 6900,
 'id': 8012672192,
 'name': '[무배/오늘출발] 압박스타킹 다리가 얇아지는 검정 학생 교복 스타킹 양말',
 'photoVideoReviewPoint': 250,
 'productNo': 7974387490,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20230201_214/1675249278596DbnqC_JPEG/76385174326132234_545259486.jpg',
 'reviewCount': 1,
 'salePrice': 14900,
 'textReviewPoint': 100}
{'averageReviewScore': 4.58,
 'categoryId': '50004163',
 'categoryName': '패션잡화>주얼리>귀걸이>진주귀걸이',
 'channelName': '퍼피띵',
 'discountedRatio': 50,
 'discountedSalePrice': 7900,
 'id': 7973376564,
 'name': '[무배/당일출고] 유기견후원 실버925 하트 오로라 펄 진주 호마이카 볼드 은침 귀걸이',
 'photoVideoReviewPoint': 250,
 'productNo': 7935235885,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20230202_28/1675317524199FfU0w_JPEG/76453419902981032_1396241603.jpg',
 'rev

{'averageReviewScore': 4.84,
 'categoryId': '50004174',
 'categoryName': '패션잡화>주얼리>목걸이>패션목걸이',
 'channelName': '퍼피띵',
 'discountedRatio': 40,
 'discountedSalePrice': 11900,
 'id': 7044991880,
 'name': '[무배/당일출고] 유기견후원 써지컬목걸이 기부 물방울 목걸이',
 'photoVideoReviewPoint': 250,
 'productNo': 7011523432,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20220826_61/1661519380627EvF1H_JPEG/62655215423944826_1290644050.jpg',
 'reviewCount': 209,
 'salePrice': 19900,
 'textReviewPoint': 100}
{'averageReviewScore': 4.91,
 'categoryId': '50004194',
 'categoryName': '패션잡화>주얼리>팔찌>패션팔찌',
 'channelName': '퍼피띵',
 'discountedRatio': 13,
 'discountedSalePrice': 12900,
 'id': 6803849415,
 'name': '유기견후원팔찌 기부팔찌 학생 커플 팔찌',
 'photoVideoReviewPoint': 250,
 'productNo': 6771467935,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20230205_171/1675599430670YVMOC_JPEG/76735326400102865_575503607.jpg',
 'reviewCount': 355,
 'salePrice': 14

{'averageReviewScore': 4.84,
 'categoryId': '50004174',
 'categoryName': '패션잡화>주얼리>목걸이>패션목걸이',
 'channelName': '퍼피띵',
 'discountedRatio': 40,
 'discountedSalePrice': 11900,
 'id': 7044991880,
 'name': '[무배/당일출고] 유기견후원 써지컬목걸이 기부 물방울 목걸이',
 'photoVideoReviewPoint': 250,
 'productNo': 7011523432,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20220826_61/1661519380627EvF1H_JPEG/62655215423944826_1290644050.jpg',
 'reviewCount': 209,
 'salePrice': 19900,
 'textReviewPoint': 100}
{'averageReviewScore': 4.91,
 'categoryId': '50004194',
 'categoryName': '패션잡화>주얼리>팔찌>패션팔찌',
 'channelName': '퍼피띵',
 'discountedRatio': 13,
 'discountedSalePrice': 12900,
 'id': 6803849415,
 'name': '유기견후원팔찌 기부팔찌 학생 커플 팔찌',
 'photoVideoReviewPoint': 250,
 'productNo': 6771467935,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20230205_171/1675599430670YVMOC_JPEG/76735326400102865_575503607.jpg',
 'reviewCount': 355,
 'salePrice': 14

{'averageReviewScore': 4.84,
 'categoryId': '50004174',
 'categoryName': '패션잡화>주얼리>목걸이>패션목걸이',
 'channelName': '퍼피띵',
 'discountedRatio': 40,
 'discountedSalePrice': 11900,
 'id': 7044991880,
 'name': '[무배/당일출고] 유기견후원 써지컬목걸이 기부 물방울 목걸이',
 'photoVideoReviewPoint': 250,
 'productNo': 7011523432,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20220826_61/1661519380627EvF1H_JPEG/62655215423944826_1290644050.jpg',
 'reviewCount': 209,
 'salePrice': 19900,
 'textReviewPoint': 100}
{'averageReviewScore': 4.91,
 'categoryId': '50004194',
 'categoryName': '패션잡화>주얼리>팔찌>패션팔찌',
 'channelName': '퍼피띵',
 'discountedRatio': 13,
 'discountedSalePrice': 12900,
 'id': 6803849415,
 'name': '유기견후원팔찌 기부팔찌 학생 커플 팔찌',
 'photoVideoReviewPoint': 250,
 'productNo': 6771467935,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20230205_171/1675599430670YVMOC_JPEG/76735326400102865_575503607.jpg',
 'reviewCount': 355,
 'salePrice': 14

{'averageReviewScore': 4.84,
 'categoryId': '50004174',
 'categoryName': '패션잡화>주얼리>목걸이>패션목걸이',
 'channelName': '퍼피띵',
 'discountedRatio': 40,
 'discountedSalePrice': 11900,
 'id': 7044991880,
 'name': '[무배/당일출고] 유기견후원 써지컬목걸이 기부 물방울 목걸이',
 'photoVideoReviewPoint': 250,
 'productNo': 7011523432,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20220826_61/1661519380627EvF1H_JPEG/62655215423944826_1290644050.jpg',
 'reviewCount': 209,
 'salePrice': 19900,
 'textReviewPoint': 100}
{'averageReviewScore': 4.91,
 'categoryId': '50004194',
 'categoryName': '패션잡화>주얼리>팔찌>패션팔찌',
 'channelName': '퍼피띵',
 'discountedRatio': 13,
 'discountedSalePrice': 12900,
 'id': 6803849415,
 'name': '유기견후원팔찌 기부팔찌 학생 커플 팔찌',
 'photoVideoReviewPoint': 250,
 'productNo': 6771467935,
 'productStatusType': 'SALE',
 'representativeImageUrl': 'https://shop-phinf.pstatic.net/20230205_171/1675599430670YVMOC_JPEG/76735326400102865_575503607.jpg',
 'reviewCount': 355,
 'salePrice': 14

In [268]:
len(item_list)

254

In [16]:
url_list3 =[
#     'https://m.smartstore.naver.com/de_lemonmarket',
#  'https://smartstore.naver.com/puffy_thing',
 'https://smartstore.naver.com/trendyttt',
 'https://smartstore.naver.com/heelaser',
 'https://smartstore.naver.com/myodeng',
 'https://smartstore.naver.com/ppottuppttu',
 'https://smartstore.naver.com/gfound',
 'https://m.smartstore.naver.com/petsdaily',
 'https://smartstore.naver.com/waterstar',
 'https://smartstore.naver.com/kongdahouse',
 'https://smartstore.naver.com/pawinhand',
 'https://smartstore.naver.com/peticion_1214',
 'https://m.smartstore.naver.com/unimpressedhusky',
 'https://smartstore.naver.com/moii',
#  'https://smartstore.naver.com/hisdog'
]

In [17]:
item_list=[]
brand_list=[]

for url in url_list3:
    res2 = smartstore_json(url) # smartstore url 로부터 json 추출
    item(res2,url) # item_list 에 각 item 별 json 데이터 dicctionary 에 추가
    brand(res2,url) # brand_list

{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 18000,
 'pimg': 'https://shop-phinf.pstatic.net/20230207_8/1675758159201Y1gBK_JPEG/76893942902989142_797488726.jpg',
 'pname': '다온 아동 유기견 후원 기부 써지컬 반지 (하트구름)',
 'psalePrice': 18000,
 'purl': 'https://smartstore.naver.com/trendyttt/products/6441450775'}
{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 17000,
 'pimg': 'https://shop-phinf.pstatic.net/20230321_27/1679331287276uBIQT_JPEG/80467067000806972_731107737.jpg',
 'pname': '다온 의미있는 아동 유기견 후원 기부 써지컬 반지 (레터링)',
 'psalePrice': 17000,
 'purl': 'https://smartstore.naver.com/trendyttt/products/6458380850'}
{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 9900,
 'pimg': 'https://shop-phinf.pstatic.ne

{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 18000,
 'pimg': 'https://shop-phinf.pstatic.net/20230207_8/1675758159201Y1gBK_JPEG/76893942902989142_797488726.jpg',
 'pname': '다온 아동 유기견 후원 기부 써지컬 반지 (하트구름)',
 'psalePrice': 18000,
 'purl': 'https://smartstore.naver.com/trendyttt/products/6441450775'}
{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 17000,
 'pimg': 'https://shop-phinf.pstatic.net/20230321_27/1679331287276uBIQT_JPEG/80467067000806972_731107737.jpg',
 'pname': '다온 의미있는 아동 유기견 후원 기부 써지컬 반지 (레터링)',
 'psalePrice': 17000,
 'purl': 'https://smartstore.naver.com/trendyttt/products/6458380850'}
{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 9900,
 'pimg': 'https://shop-phinf.pstatic.ne

{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 18000,
 'pimg': 'https://shop-phinf.pstatic.net/20230207_8/1675758159201Y1gBK_JPEG/76893942902989142_797488726.jpg',
 'pname': '다온 아동 유기견 후원 기부 써지컬 반지 (하트구름)',
 'psalePrice': 18000,
 'purl': 'https://smartstore.naver.com/trendyttt/products/6441450775'}
{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 17000,
 'pimg': 'https://shop-phinf.pstatic.net/20230321_27/1679331287276uBIQT_JPEG/80467067000806972_731107737.jpg',
 'pname': '다온 의미있는 아동 유기견 후원 기부 써지컬 반지 (레터링)',
 'psalePrice': 17000,
 'purl': 'https://smartstore.naver.com/trendyttt/products/6458380850'}
{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 9900,
 'pimg': 'https://shop-phinf.pstatic.ne

{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 18000,
 'pimg': 'https://shop-phinf.pstatic.net/20230207_8/1675758159201Y1gBK_JPEG/76893942902989142_797488726.jpg',
 'pname': '다온 아동 유기견 후원 기부 써지컬 반지 (하트구름)',
 'psalePrice': 18000,
 'purl': 'https://smartstore.naver.com/trendyttt/products/6441450775'}
{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 17000,
 'pimg': 'https://shop-phinf.pstatic.net/20230321_27/1679331287276uBIQT_JPEG/80467067000806972_731107737.jpg',
 'pname': '다온 의미있는 아동 유기견 후원 기부 써지컬 반지 (레터링)',
 'psalePrice': 17000,
 'purl': 'https://smartstore.naver.com/trendyttt/products/6458380850'}
{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 9900,
 'pimg': 'https://shop-phinf.pstatic.ne

{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 18000,
 'pimg': 'https://shop-phinf.pstatic.net/20230207_8/1675758159201Y1gBK_JPEG/76893942902989142_797488726.jpg',
 'pname': '다온 아동 유기견 후원 기부 써지컬 반지 (하트구름)',
 'psalePrice': 18000,
 'purl': 'https://smartstore.naver.com/trendyttt/products/6441450775'}
{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 17000,
 'pimg': 'https://shop-phinf.pstatic.net/20230321_27/1679331287276uBIQT_JPEG/80467067000806972_731107737.jpg',
 'pname': '다온 의미있는 아동 유기견 후원 기부 써지컬 반지 (레터링)',
 'psalePrice': 17000,
 'purl': 'https://smartstore.naver.com/trendyttt/products/6458380850'}
{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 9900,
 'pimg': 'https://shop-phinf.pstatic.ne

{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 18000,
 'pimg': 'https://shop-phinf.pstatic.net/20230207_8/1675758159201Y1gBK_JPEG/76893942902989142_797488726.jpg',
 'pname': '다온 아동 유기견 후원 기부 써지컬 반지 (하트구름)',
 'psalePrice': 18000,
 'purl': 'https://smartstore.naver.com/trendyttt/products/6441450775'}
{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 17000,
 'pimg': 'https://shop-phinf.pstatic.net/20230321_27/1679331287276uBIQT_JPEG/80467067000806972_731107737.jpg',
 'pname': '다온 의미있는 아동 유기견 후원 기부 써지컬 반지 (레터링)',
 'psalePrice': 17000,
 'purl': 'https://smartstore.naver.com/trendyttt/products/6458380850'}
{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 9900,
 'pimg': 'https://shop-phinf.pstatic.ne

{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 18000,
 'pimg': 'https://shop-phinf.pstatic.net/20230207_8/1675758159201Y1gBK_JPEG/76893942902989142_797488726.jpg',
 'pname': '다온 아동 유기견 후원 기부 써지컬 반지 (하트구름)',
 'psalePrice': 18000,
 'purl': 'https://smartstore.naver.com/trendyttt/products/6441450775'}
{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 17000,
 'pimg': 'https://shop-phinf.pstatic.net/20230321_27/1679331287276uBIQT_JPEG/80467067000806972_731107737.jpg',
 'pname': '다온 의미있는 아동 유기견 후원 기부 써지컬 반지 (레터링)',
 'psalePrice': 17000,
 'purl': 'https://smartstore.naver.com/trendyttt/products/6458380850'}
{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 9900,
 'pimg': 'https://shop-phinf.pstatic.ne

{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 18000,
 'pimg': 'https://shop-phinf.pstatic.net/20230207_8/1675758159201Y1gBK_JPEG/76893942902989142_797488726.jpg',
 'pname': '다온 아동 유기견 후원 기부 써지컬 반지 (하트구름)',
 'psalePrice': 18000,
 'purl': 'https://smartstore.naver.com/trendyttt/products/6441450775'}
{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 17000,
 'pimg': 'https://shop-phinf.pstatic.net/20230321_27/1679331287276uBIQT_JPEG/80467067000806972_731107737.jpg',
 'pname': '다온 의미있는 아동 유기견 후원 기부 써지컬 반지 (레터링)',
 'psalePrice': 17000,
 'purl': 'https://smartstore.naver.com/trendyttt/products/6458380850'}
{'bname': '다온 :DAON',
 'pbaseFee': 3000,
 'pcategory1': '패션잡화>주얼리>반지>패션반지',
 'pcategory2': '패션반지',
 'pdiscountedRatio': 0,
 'pdiscountedSalePrice': 9900,
 'pimg': 'https://shop-phinf.pstatic.ne

In [29]:
cate_list1 = []
cate_list2 = []
for item in item_list:
    cate1 = item['pcategory1']
    cate2 = item['pcategory2']
    if cate1 in cate_list1:
        cate_list1.append(cate1)
    if cate2 in cate_list2:
        cate_list2.append(cate2)

In [30]:
cate_list1 = []
cate_list2 = []
cnt1 = 1
cnt2= 1
for item in item_list:
    cate1 = item['pcategory1']
    cate2 = item['pcategory2']
    if cate1 not in cate_list1:
        cate_list1.append(cate1)
        cnt1 += 1
    if cate2 not in cate_list2:
        cate_list2.append(cate2)
        cnt2 +=1

In [32]:
cate_list1

['패션잡화>주얼리>반지>패션반지',
 '생활/건강>주방용품>잔/컵>텀블러',
 '디지털/가전>휴대폰액세서리>휴대폰케이스>기타케이스',
 '디지털/가전>음향가전>이어폰/헤드폰액세서리>케이스/파우치',
 '패션잡화>주얼리>목걸이>패션목걸이',
 '디지털/가전>휴대폰액세서리>휴대폰거치대',
 '생활/건강>문구/사무용품>노트/수첩>메모지',
 '디지털/가전>휴대폰액세서리>기타휴대폰액세서리',
 '패션잡화>주얼리>팔찌>패션팔찌',
 '생활/건강>주방용품>잔/컵>머그',
 '패션잡화>패션소품>한복소품',
 '패션잡화>패션소품>키홀더',
 '패션잡화>헤어액세서리>헤어핀',
 '생활/건강>종교>천주교용품',
 '생활/건강>반려동물>기타반려동물용품',
 '생활/건강>문구/사무용품>필기도구>볼펜',
 '생활/건강>문구/사무용품>스티커/테이프>스티커',
 '생활/건강>생활용품>제습/방향/탈취>섬유탈취스프레이',
 '생활/건강>주방용품>잔/컵>유리컵',
 '패션잡화>주얼리>발찌>패션발찌',
 '생활/건강>반려동물>강아지 장난감/훈련>장난감/토이',
 '생활/건강>반려동물>패션용품>스카프/목도리/케이프',
 '패션잡화>헤어액세서리>헤어액세서리소품',
 '패션잡화>패션소품>기타패션소품',
 '생활/건강>반려동물>패션용품>헤어핀/주얼리',
 '생활/건강>반려동물>고양이 장난감>낚시/막대',
 '도서>잡지>여행/취미/스포츠>기타',
 '도서>유아>유아학습>기타',
 '생활/건강>반려동물>패션용품>기타액세서리',
 '생활/건강>반려동물>야외용품>가슴줄',
 '출산/육아>인형>봉제인형',
 '생활/건강>반려동물>리빙용품>쿠션/방석',
 '패션잡화>패션소품>브로치>패션브로치',
 '패션잡화>여성가방>숄더백',
 '생활/건강>반려동물>야외용품>리드줄',
 '도서>건강/취미>취미/레저>반려동물',
 '패션잡화>주얼리>팔찌>은팔찌',
 '생활/건강>종교>불교용품',
 '패션잡화>주얼리>목걸이>진주목걸이',
 '패션잡화>주얼리>반지>은반지',
 '생활/건강>반려동물>강아지 건강/관리용품>눈/귀 관리

In [27]:
cate_list2

['패션반지',
 '텀블러',
 '기타케이스',
 '케이스/파우치',
 '패션목걸이',
 '휴대폰거치대',
 '메모지',
 '기타휴대폰액세서리',
 '패션팔찌',
 '머그',
 '한복소품',
 '키홀더',
 '헤어핀',
 '천주교용품',
 '기타반려동물용품',
 '볼펜',
 '스티커',
 '섬유탈취스프레이',
 '유리컵',
 '패션발찌',
 '장난감/토이',
 '스카프/목도리/케이프',
 '헤어액세서리소품',
 '기타패션소품',
 '헤어핀/주얼리',
 '낚시/막대',
 '기타',
 '기타액세서리',
 '가슴줄',
 '봉제인형',
 '쿠션/방석',
 '패션브로치',
 '숄더백',
 '리드줄',
 '반려동물',
 '은팔찌',
 '불교용품',
 '진주목걸이',
 '은반지',
 '눈/귀 관리용품']